3. **Dificultad Media:** (i) Modifica el algoritmo de Adams-Bashforth para considerar paso variable. Para esto es necesario que se calcule la sexta derivada de la función. Esto lo puedes hacer con un método numérico (re cuerda que los métodos numéricos fallan si usas $h << 1$), usando la técnica de los coefcientes para obtener la derivada en un solo paso (Tarea de Cálculo problema 1 y 2), o bien usando SymPy. Si es usando SymPy, asegúrate primero de obtener la derivada una sola vez en tu función y transformar esa función en una función numérica (no vayas a obtener la derivada dentro de algún ciclo for o while). 

    (ii) Prueba este algoritmo para un sistema de ecuaciones diferenciales (el que quieras, pero del cual conozcas su solución).

In [1]:
function raiz_n(x, a = x, n=1, ϵ = 0.001)
    contador = 0
    while abs(a^n - x) > ϵ && contador <1000
        contador += 1
        a = (n-1)/n * a + x/(n*a^(n-1))
    end
    return a
end 

raiz_n (generic function with 4 methods)

In [2]:
raiz_n.([0.0002,2,3],1.0,5)

3-element Vector{Float64}:
 0.22160736937126738
 1.148728886527325
 1.2457341658776022

In [17]:
using TaylorSeries

function dervN(f,a,n=1) ## Defino una función que calcula la derivad de una función empleando TaylorSeries.
    t = Taylor1(eltype(a), n)
    fT = f(a+t)
    R= fT[n]*factorial(n)
    return R
end

Dn(f,x,n)=[dervN(f[i],x,n) for i in 1:2]

Dn (generic function with 1 method)

In [18]:
p = [vx->(vx)^7,vy->(vy)^8];

In [19]:
Dn(p,1,6)

2-element Vector{Int64}:
  5040
 20160

In [20]:
function RK4(f, x0, t, h = 0.01) # es de orden 4, así que en realidad el error en cada paso será 10^-8
    t0 = 0
    n = floor(Int, (t/h))
    X = [zero(x0) for i in 1:n+1]
    X[1] = x0
    for i in 2:n+1
        k1 = f(x0, t0)
        k2 = f(x0 .+ h*k1./2, t0 + h/2)
        k3 = f(x0 .+ h*k2./2, t0 + h/2)
        k4 = f(x0 .+ h*k3, t0 + h)  
        x0 = x0 .+ h/6 .* (k1 .+ 2*k2 .+ 2*k3 .+ k4) 
        t0 += h
        X[i] = x0
    end
    return X
end 

RK4 (generic function with 2 methods)

In [21]:
function Admas_Bashforth(f, x0, t, h0 = 0.01)
    t0 = 0
    n = floor(Int, t/h0)
    X = [zero(x0) for i in 1:n+1]
    X[1:5] = RK4(f, x0, 4*h0, h0)
    for i in 5:n
        X[i+1] = X[i] .+ h0/720 .* (1901*f(X[i], i*h0) .- 2774*f(X[i-1], (i-1)*h0) .+ 2616*f(X[i-2], (i-2)*h0) .- 1274*f(X[i-3], (i-3)*h0) .+ 251*f(X[i-4], (i-4)*h0)) 
    end
    return X
end

Admas_Bashforth (generic function with 2 methods)

In [22]:
# G = 1
# m1 = 1
# m2 = 1
# g(x, t) = [-G*m1*m2*x[3]/(x[3]^2+x[4]^2)^(3/2), -G*m1*m2*x[4]/(x[3]^2+x[4]^2)^(3/2), x[1], x[2]]
# x_0 = [1,0.0,0.0,10]
# X = Admas_Bashforth(g, x_0, 2000, 0.01)

In [26]:
function Admas_BashforthV(f, x0, t, h0 = 0.01)
    t0 = 0
    df(t)= Dn(f,t,6)
    h= h0/raiz_n.((df(t)),1.0,5)
    X=[zero(x0)]
    x[1]=x0
    contador = 0
    T = [zero(h0)]
    
    while t0<t && contador <1000
    contador +=1
    n = floor(Int, t/h)
    X = [zero(x0) for i in 1:n+1]
    X[1:5] = RK4(f, x0, 4*h, h)
    for i in 5:n
        X[i+1] = X[i] .+ h/720 .* (1901*f(X[i], i*h) .- 2774*f(X[i-1], (i-1)*h) .+ 2616*f(X[i-2], (i-2)*h) .- 1274*f(X[i-3], (i-3)*h) .+ 251*f(X[i-4], (i-4)*h)) 
    end
    return X
    
    end
    
    return T,X
end

Admas_BashforthV (generic function with 2 methods)

Por ejemplo, resolvamos la ecuación de un oscilador armónico $\ddot{x}(t) = \dot{v}(t) = -kx$. Esta ecuación de segundo grado la podemos ver en realidad como un sistema de ecuaciones de primer grado:  

$$\dot{v}(t) = -kx$$
$$\dot{x}(t) = v$$

In [31]:
k = 2.2
f(x)=-k*x
h(v)=v
g(x,t) = [f, h]
x_0 = [0, 13.1]
t = 20
X = Admas_BashforthV(g, x_0, t)

LoadError: MethodError: no method matching getindex(::typeof(g), ::Int64)

7.**Muy Fácil:** Revisa aquí: https://modelo.covid19.cdmx.gob.mx/modelo-epidemico el modelo epidemeológico de la CDMX. Escribe las ecuaciones, con los parámetros que ahí dan y resuelve el modelo con algún solucionador "bueno". 

**Nota:** No necesitas responder, pero piensa al respecto. ¿La simulación se acerca algo a lo que sucedió en la ciudad? ¿Al menos se acerca a los primeros meses? ¿Cómo se podría mejorar este modelo?

$$\begin{split}
\dot S & = - \left(\frac{R_0}{D_{infect}} \right) IS \\
\dot E & = \left(\frac{R_0}{D_{infect}} \right) IS - \left(\frac{1}{D_{incub}} \right)E\\
\dot I & = \left(\frac{1}{D_{incub}} \right)E- \left(\frac{1}{D_{infect}} \right)I\\
\dot L & = (1-p_{grave})\left(\frac{1}{D_{infect}} \right)I - \left(\frac{1}{D_{RL}} \right)L \\
\dot G & = p_{grave} \left(\frac{1}{D_{infect}} \right)I - \left(\frac{1}{D_{hosp}} \right)G\\
\dot H & = \left(\frac{1}{D_{hosp}} \right)G - (1-p_{ICU})\left(\frac{1}{D_{RH}} \right)H - p_{ICU} \left(\frac{1}{D_{ICU}} \right)H\\
\dot (ICU) & = p_{ICU}\left(\frac{1}{D_{ICU}} \right)H - (1-p_{M})\left(\frac{1}{D_{RICU}} \right)ICU - p_{M} \left(\frac{1}{D_{M}} \right)ICU\\
\dot R & = \left(\frac{1}{D_{RL}} \right)L + (1-p_{ICU}) \left(\frac{1}{D_{RH}} \right)H + (1-p_{M})\left(\frac{1}{D_{RICU}} \right)ICU\\
\dot M & = p_{M}\left(\frac{1}{D_{M}} \right)ICU
\end{split}$$

$$S+E+I+L+G+H+ICU+R+M=1$$

In [2]:
using Plots

In [3]:
function RK4(f, x0, t, h = 0.01) # es de orden 4, así que en realidad el error en cada paso será 10^-8
    t0 = 0
    n = floor(Int, (t/h))
    X = [zero(x0) for i in 1:n+1]
    X[1] = x0
    for i in 2:n+1
        k1 = f(x0, t0)
        k2 = f(x0 .+ h*k1./2, t0 + h/2)
        k3 = f(x0 .+ h*k2./2, t0 + h/2)
        k4 = f(x0 .+ h*k3, t0 + h)  
        x0 = x0 .+ h/6 .* (k1 .+ 2*k2 .+ 2*k3 .+ k4) 
        t0 += h
        X[i] = x0
    end
    return X
end 

RK4 (generic function with 2 methods)

In [4]:
#multiplicador por sobreporte 2
N=22
D_infect=2.9
D_incub=5.2
p_grave=(13.80)*N/100
D_RL=14
D_hosp=4
p_ICU=(5)*N/100
D_RH=12
p_M=3*N/100
D_ICU=1
D_RICU=7
D_M=8
R_0=2.83

2.83

In [5]:
g(x,t) = [-(R_0/D_infect)*x[3]*x[1], (R_0/D_infect)*x[3]*x[1] - (1/D_incub)*x[2], (1/D_incub)*x[2]-(1/D_infect)*x[3],(1-p_grave)*(1/D_infect)*x[3]-(1/D_RL)*x[4],(p_grave)*(1/D_infect)*x[3]-(1/D_hosp)*x[5], (1/D_hosp)*x[5]-(1-p_ICU)*(1/D_RH)x[6]-p_ICU+(1/D_ICU)*x[6],(p_ICU)*(1/D_ICU)*x[6]-(1-p_M)*(1/D_RICU)*x[7]-(p_M)*(1/D_M)*x[7], (1/D_RL)*x[4]+(1-p_ICU)*(1/D_RH)*x[6]+(1-p_M)*(1/D_RICU)*x[7], (p_M)*(1/D_M)*x[7]]


x_0 = [N,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0]

X = RK4(g, x_0, 5, 0.1) 

t = range(0,stop=150,length=length(X))

plotly()
plot(t, [X[i][1] for i in 1:length(X)], label = "Susceptibles")
plot!(t, [X[i][2] for i in 1:length(X)], label = "Expuestos", legend = :outertopright, aspect_ratio=1)
plot!(t, [X[i][3] for i in 1:length(X)], label = "Infectados")
plot!(t, [X[i][4] for i in 1:length(X)], label = "C. leves")
plot!(t, [X[i][5] for i in 1:length(X)], label = "C. graves")
plot!(t, [X[i][6] for i in 1:length(X)], label = "Hospitalizados")
plot!(t, [X[i][7] for i in 1:length(X)], label = "ICU")
plot!(t, [X[i][8] for i in 1:length(X)], label = "Recuperados")
plot!(t, [X[i][9] for i in 1:length(X)], label = "Muertos")


┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\sealp\.julia\packages\Plots\Awg62\src\backends.jl:377
